In [6]:
from GSNN import utils
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
import torch

from GSNN.GSNNmodel import GSNN, GCN
from GSNN.GSNNTrain import GSNNTrainer, GCNTrainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
dataset = 'cora'# dataset = {'cora', 'citeseer', 'pubmed'}
seed = 15
ratio = 0.01 # ratio of training set
unlabeled_share = 0.8
train_share = ratio 
val_share = 1 - unlabeled_share - train_share
weight_decay = 5e-4
lr = 1e-2
hidden_dim = 16
attack = None
epochs = 200
earlystopping = False

In [8]:
_A_obs, _X_obs, _z_obs = utils.load_npz('data/' + dataset + '.npz')

if _X_obs is None:
    _X_obs = sp.eye(_A_obs.shape[0]).tocsr()
lcc = utils.largest_connected_components(_A_obs)


if attack=='meta_self':
    _A_obs = sp.csr_matrix(np.load('data/' + dataset + '_0.05edges_Meta-Self.npy'))
elif attack=='meta_train':
    _A_obs = sp.csr_matrix(np.load('data/' + dataset + '_0.05edges_Meta-Train.npy'))
elif attack=='minmax':
    _A_obs = sp.csr_matrix(np.load('data/' + dataset + '_0.05_minmax.npy'))
else:
    _A_obs = _A_obs[lcc][:,lcc]

_A_obs = _A_obs + _A_obs.T
_A_obs[_A_obs > 1] = 1


_A_obs.setdiag(0)
_A_obs = _A_obs.astype("float32")
_A_obs.eliminate_zeros()
_X_obs = _X_obs.astype("float32")

_X_obs = _X_obs[lcc]
_z_obs = _z_obs[lcc]
_N = _A_obs.shape[0]
_K = _z_obs.max()+1
_Z_obs = np.eye(_K)[_z_obs]

#normalized adj
_An = utils.preprocess_graph(_A_obs)
degrees = _A_obs.sum(0).A1

np.random.seed(seed)

split_train, split_val, split_test = utils.train_val_test_split_tabular(np.arange(_N),
                                                                    train_size=train_share,
                                                                    val_size=val_share,
                                                                    test_size=unlabeled_share,
                                                                    stratify=_z_obs)
print('Number of trainging set: {}'.format(split_train.shape[0]))
split_unlabeled = np.union1d(split_val, split_test)

utils.set_seed(42, torch.cuda.is_available())
adj = utils.sparse_mx_to_torch_sparse_tensor(_An).float()
adj_cuda = adj.cuda()

feature_cuda, label_cuda, idx_train_cuda, idx_val_cuda, idx_test_cuda = utils.convert_to_Tensor([_X_obs, _Z_obs, split_train, split_val, split_test])

Selecting 1 largest connected components


e:\anaconda\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Number of trainging set: 24


In [11]:
# An instantiation of GSNN-M model
x_dim = feature_cuda.shape[1]
y_dim = label_cuda.shape[1]
h_dim, z_dim, r_dim = utils.get_h_dim(hidden_dim)

gcn = GCN(x_dim, h_dim, y_dim, True, False)
gsnn = GSNN(x_dim, y_dim, h_dim, r_dim, z_dim, gcn)

optimizer = torch.optim.Adam(gsnn.parameters(), lr=lr, weight_decay=weight_decay)
gsnntrainer = GSNNTrainer(gsnn, optimizer)

In [12]:
res = gsnntrainer.train(adj_cuda, feature_cuda, label_cuda, idx_train_cuda, idx_val_cuda, idx_test_cuda, epochs, earlystopping=earlystopping)

Epoch: 0, Train_loss: 14.131916999816895.
--------------------------------------------------
Epoch: 10, Train_loss: 4.617766380310059.
--------------------------------------------------
Epoch: 20, Train_loss: 3.630624532699585.
--------------------------------------------------
Epoch: 30, Train_loss: 2.9686386585235596.
--------------------------------------------------
Epoch: 40, Train_loss: 2.415328025817871.
--------------------------------------------------
Epoch: 50, Train_loss: 2.109048843383789.
--------------------------------------------------
Epoch: 60, Train_loss: 2.145073652267456.
--------------------------------------------------
Epoch: 70, Train_loss: 1.836736798286438.
--------------------------------------------------
Epoch: 80, Train_loss: 1.7633899450302124.
--------------------------------------------------
Epoch: 90, Train_loss: 1.6819504499435425.
--------------------------------------------------
Epoch: 100, Train_loss: 1.6895185708999634.
-----------------------